In [1]:
import gym
import gym_watten
import numpy as np
from time import sleep


In [2]:
%load_ext line_profiler

In [3]:
env = gym.make("Watten-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: Environment '<class 'gym_watten.envs.watten_env.WattenEnv'>' has deprecated methods. Compatibility code invoked.


In [6]:
obs, is_done = env.reset(), False

In [ ]:
state = env.get_state()
org_obs = obs

In [5]:
while not is_done:
    env.render('human')    
    obs, rew, is_done, _ = env.step(np.argmax(obs))
    print(rew)
    sleep(1)

[] False 5
[0, 1] False 12
[] False 18
[0, 1] False 15
[] False 21
[0, 6] True 25


In [ ]:
[card.id for card in env.cards]

In [ ]:
env.set_state(state)
is_done = False
obs = org_obs


In [86]:
sample_outputs = np.zeros((11959, 5))
sample_inputs = np.zeros((11959, 8), dtype=int)
next_index = 0

In [87]:
def search(obs):
    global sample_outputs, sample_inputs, next_index
    state = env.get_state()
    n = len(env.players[env.current_player].hand_cards)   
    p = 0
    local_index = next_index
    next_index += 1
    
    for i in range(n):
        sample_inputs[local_index][i] = env.players[env.current_player].hand_cards[i].id
        obs, rew, is_done, _ = env.step(sample_inputs[local_index][i])
        if is_done:
            sample_outputs[local_index][i] = (rew[0] > 0) / n
        else:
            sample_outputs[local_index][i] = (1 - search(obs)) / n
        env.set_state(state)
        p += sample_outputs[local_index][i]
        
    if env.table_card is not None:
        sample_inputs[local_index][5] = env.table_card.id
    sample_inputs[local_index][6] = env.players[env.current_player].tricks
    sample_inputs[local_index][7] = env.players[1 - env.current_player].tricks
    return p 

In [88]:
for i in range(1):
    obs = env.reset()
    search(obs)
next_index

8685

In [89]:
sample_inputs

array([[11,  9, 27, ...,  0,  0,  0],
       [19, 22,  4, ..., 11,  0,  0],
       [ 9, 27, 23, ...,  0,  1,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [95]:
sample_outputs

1.0

In [ ]:
env.render('human')    

In [ ]:
obs, rew, is_done, _ = env.step(2)

In [78]:
%lprun -f search search(obs)